# Exploração e Prototipagem - AUDITORIA360

## 📋 Visão Geral

Este notebook é utilizado para a exploração de dados e prototipagem de modelos relacionados à auditoria de folha de pagamento. Aqui, realizaremos análises exploratórias, visualizações e testes de diferentes abordagens de modelagem.

## 🎯 Objetivos

- Análise exploratória de dados de folha de pagamento
- Identificação de padrões e anomalias
- Prototipagem de modelos de machine learning
- Validação de hipóteses de auditoria

## 📚 Pré-requisitos

- Python 3.8+
- Bibliotecas: pandas, numpy, matplotlib, seaborn, scikit-learn
- Dados de folha de pagamento processados

## 🚀 Como Usar

1. Execute as células sequencialmente
2. Certifique-se de que os dados estão disponíveis no diretório `../data/processed/`
3. Ajuste os parâmetros conforme necessário
4. Analise os resultados e visualizações geradas

## 🔧 Configuração do Ambiente

Importação das bibliotecas necessárias e configuração inicial do ambiente de análise.

In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path

# Suprimir warnings desnecessários
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Configurações do pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Ambiente configurado com sucesso!")
print(f"📦 Pandas versão: {pd.__version__}")
print(f"📊 Numpy versão: {np.__version__}")

## 📂 Carregamento e Verificação dos Dados

Carregamento dos dados de folha de pagamento e verificação inicial da estrutura.

In [ ]:
# Definir caminhos dos dados
DATA_PATH = Path('../data/processed')
RAW_DATA_PATH = Path('../data/raw')

# Verificar se os diretórios existem
if not DATA_PATH.exists():
    print("⚠️  Diretório de dados processados não encontrado.")
    print("📝 Criando dados de exemplo para demonstração...")
    
    # Criar dados de exemplo para demonstração
    np.random.seed(42)
    n_employees = 1000
    
    df_folha = pd.DataFrame({
        'funcionario_id': range(1, n_employees + 1),
        'nome': [f'Funcionário {i}' for i in range(1, n_employees + 1)],
        'cpf': [f'{np.random.randint(10000000000, 99999999999)}' for _ in range(n_employees)],
        'salario_base': np.random.normal(5000, 2000, n_employees).clip(min=1320),
        'horas_extras': np.random.exponential(10, n_employees),
        'descontos': np.random.normal(500, 200, n_employees).clip(min=0),
        'salario_liquido': None,
        'departamento': np.random.choice(['TI', 'RH', 'Financeiro', 'Operacional', 'Vendas'], n_employees),
        'cargo': np.random.choice(['Analista', 'Coordenador', 'Gerente', 'Diretor', 'Assistente'], n_employees),
        'data_admissao': pd.date_range('2020-01-01', '2024-12-31', periods=n_employees),
        'status': np.random.choice(['Ativo', 'Inativo', 'Férias'], n_employees, p=[0.85, 0.10, 0.05])
    })
    
    # Calcular salário líquido
    df_folha['salario_liquido'] = df_folha['salario_base'] + df_folha['horas_extras'] - df_folha['descontos']
    
    print("✅ Dados de exemplo criados com sucesso!")
else:
    try:
        # Tentar carregar dados reais
        df_folha = pd.read_csv(DATA_PATH / 'dados_folha.csv')
        print("✅ Dados carregados com sucesso!")
    except FileNotFoundError:
        print("⚠️  Arquivo dados_folha.csv não encontrado. Usando dados de exemplo.")
        # Usar lógica de dados de exemplo aqui também

# Informações básicas sobre o dataset
print(f"\n📊 Informações do Dataset:")
print(f"   • Número de registros: {len(df_folha):,}")
print(f"   • Número de colunas: {len(df_folha.columns)}")
print(f"   • Período dos dados: {df_folha['data_admissao'].min()} a {df_folha['data_admissao'].max()}")

# Visualizar primeiras linhas
df_folha.head()

## 🔍 Análise Exploratória dos Dados

Nesta seção, realizaremos uma análise exploratória dos dados para entender melhor as características e padrões presentes na folha de pagamento.

In [ ]:
# Análise descritiva dos dados
print("📊 ESTATÍSTICAS DESCRITIVAS")
print("=" * 50)

# Informações gerais
print("\n🔢 Informações Gerais:")
print(df_folha.info())

# Estatísticas das colunas numéricas
print("\n📈 Estatísticas das Variáveis Numéricas:")
numeric_cols = df_folha.select_dtypes(include=[np.number]).columns
display(df_folha[numeric_cols].describe())

# Análise de valores únicos para variáveis categóricas
print("\n📋 Análise de Variáveis Categóricas:")
categorical_cols = df_folha.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if col not in ['nome', 'cpf']:  # Excluir colunas com muitos valores únicos
        print(f"\n{col.upper()}:")
        print(df_folha[col].value_counts())

In [ ]:
# Verificação de dados faltantes e outliers
print("🔍 VERIFICAÇÃO DE QUALIDADE DOS DADOS")
print("=" * 50)

# Valores faltantes
missing_data = df_folha.isnull().sum()
if missing_data.sum() > 0:
    print("\n⚠️  Valores Faltantes Encontrados:")
    missing_df = pd.DataFrame({
        'Coluna': missing_data.index,
        'Valores Faltantes': missing_data.values,
        'Percentual': (missing_data.values / len(df_folha) * 100).round(2)
    })
    display(missing_df[missing_df['Valores Faltantes'] > 0])
else:
    print("\n✅ Nenhum valor faltante encontrado!")

# Detecção de outliers usando IQR
print("\n🎯 Detecção de Outliers (Método IQR):")
for col in ['salario_base', 'horas_extras', 'descontos', 'salario_liquido']:
    Q1 = df_folha[col].quantile(0.25)
    Q3 = df_folha[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df_folha[(df_folha[col] < (Q1 - 1.5 * IQR)) | (df_folha[col] > (Q3 + 1.5 * IQR))]
    print(f"   • {col}: {len(outliers)} outliers ({len(outliers)/len(df_folha)*100:.2f}%)")

## 📊 Visualizações Exploratórias

Criação de visualizações para identificar padrões, tendências e possíveis anomalias nos dados.

In [ ]:
# Configurar layout de subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Distribuições das Principais Variáveis Numéricas', fontsize=16, fontweight='bold')

# Distribuição do salário base
axes[0, 0].hist(df_folha['salario_base'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Distribuição do Salário Base')
axes[0, 0].set_xlabel('Salário Base (R$)')
axes[0, 0].set_ylabel('Frequência')
axes[0, 0].grid(True, alpha=0.3)

# Distribuição das horas extras
axes[0, 1].hist(df_folha['horas_extras'], bins=30, alpha=0.7, color='lightgreen', edgecolor='black')
axes[0, 1].set_title('Distribuição das Horas Extras')
axes[0, 1].set_xlabel('Horas Extras (R$)')
axes[0, 1].set_ylabel('Frequência')
axes[0, 1].grid(True, alpha=0.3)

# Distribuição dos descontos
axes[1, 0].hist(df_folha['descontos'], bins=30, alpha=0.7, color='salmon', edgecolor='black')
axes[1, 0].set_title('Distribuição dos Descontos')
axes[1, 0].set_xlabel('Descontos (R$)')
axes[1, 0].set_ylabel('Frequência')
axes[1, 0].grid(True, alpha=0.3)

# Distribuição do salário líquido
axes[1, 1].hist(df_folha['salario_liquido'], bins=30, alpha=0.7, color='gold', edgecolor='black')
axes[1, 1].set_title('Distribuição do Salário Líquido')
axes[1, 1].set_xlabel('Salário Líquido (R$)')
axes[1, 1].set_ylabel('Frequência')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Análise por departamento e cargo
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Distribuição por departamento
dept_counts = df_folha['departamento'].value_counts()
axes[0].pie(dept_counts.values, labels=dept_counts.index, autopct='%1.1f%%', startangle=90)
axes[0].set_title('Distribuição de Funcionários por Departamento')

# Salário médio por cargo
salary_by_position = df_folha.groupby('cargo')['salario_liquido'].mean().sort_values(ascending=True)
axes[1].barh(salary_by_position.index, salary_by_position.values, color='lightcoral')
axes[1].set_title('Salário Líquido Médio por Cargo')
axes[1].set_xlabel('Salário Líquido Médio (R$)')

# Adicionar valores nas barras
for i, v in enumerate(salary_by_position.values):
    axes[1].text(v + 50, i, f'R$ {v:,.0f}', va='center')

plt.tight_layout()
plt.show()

## 🤖 Prototipagem de Modelos de Machine Learning

Nesta seção, testaremos diferentes modelos de machine learning para identificar possíveis riscos e anomalias na folha de pagamento.

In [ ]:
# Preparação dos dados para modelagem
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cluster import KMeans

print("🔧 PREPARAÇÃO DOS DADOS PARA MODELAGEM")
print("=" * 50)

# Criar variável target para detecção de anomalias
# Consideramos como anomalia salários muito altos ou muito baixos para o cargo
df_model = df_folha.copy()

# Encoding de variáveis categóricas
le_dept = LabelEncoder()
le_cargo = LabelEncoder()
le_status = LabelEncoder()

df_model['departamento_encoded'] = le_dept.fit_transform(df_model['departamento'])
df_model['cargo_encoded'] = le_cargo.fit_transform(df_model['cargo'])
df_model['status_encoded'] = le_status.fit_transform(df_model['status'])

# Criar features adicionais
df_model['tempo_empresa'] = (pd.Timestamp.now() - pd.to_datetime(df_model['data_admissao'])).dt.days
df_model['razao_extras_base'] = df_model['horas_extras'] / df_model['salario_base']
df_model['razao_desconto_base'] = df_model['descontos'] / df_model['salario_base']

# Selecionar features para modelagem
features = ['salario_base', 'horas_extras', 'descontos', 'departamento_encoded', 
           'cargo_encoded', 'status_encoded', 'tempo_empresa', 'razao_extras_base', 'razao_desconto_base']

X = df_model[features]

print(f"✅ Features selecionadas: {len(features)}")
print(f"📊 Shape dos dados: {X.shape}")
print(f"🎯 Features: {features}")

In [ ]:
# Modelo 1: Detecção de Anomalias usando Isolation Forest
print("🌲 MODELO 1: ISOLATION FOREST (DETECÇÃO DE ANOMALIAS)")
print("=" * 60)

# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Treinar Isolation Forest
iso_forest = IsolationForest(contamination=0.1, random_state=42)
anomaly_labels = iso_forest.fit_predict(X_scaled)

# Adicionar labels ao dataframe
df_model['anomalia'] = anomaly_labels
df_model['anomalia_label'] = df_model['anomalia'].map({1: 'Normal', -1: 'Anomalia'})

# Análise dos resultados
anomaly_counts = df_model['anomalia_label'].value_counts()
print(f"\n📊 Resultados da Detecção de Anomalias:")
print(f"   • Normal: {anomaly_counts['Normal']} ({anomaly_counts['Normal']/len(df_model)*100:.1f}%)")
print(f"   • Anomalias: {anomaly_counts['Anomalia']} ({anomaly_counts['Anomalia']/len(df_model)*100:.1f}%)")

# Visualizar anomalias
plt.figure(figsize=(12, 8))
scatter = plt.scatter(df_model['salario_base'], df_model['salario_liquido'], 
                     c=df_model['anomalia'], cmap='coolwarm', alpha=0.6)
plt.xlabel('Salário Base (R$)')
plt.ylabel('Salário Líquido (R$)')
plt.title('Detecção de Anomalias: Salário Base vs Salário Líquido')
plt.colorbar(scatter, label='Anomalia (-1) / Normal (1)')
plt.grid(True, alpha=0.3)
plt.show()

# Analisar características das anomalias
print("\n🔍 Características das Anomalias Detectadas:")
anomalias = df_model[df_model['anomalia'] == -1]
print(f"\n📈 Estatísticas das Anomalias:")
print(anomalias[['salario_base', 'horas_extras', 'descontos', 'salario_liquido']].describe())

In [ ]:
# Modelo 2: Clustering para Segmentação de Funcionários
print("🎯 MODELO 2: K-MEANS CLUSTERING (SEGMENTAÇÃO)")
print("=" * 50)

# Determinar número ótimo de clusters usando o método do cotovelo
inertias = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)

# Plotar método do cotovelo
plt.figure(figsize=(10, 6))
plt.plot(k_range, inertias, marker='o', linewidth=2, markersize=8)
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Inércia')
plt.title('Método do Cotovelo para Determinação do K Ótimo')
plt.grid(True, alpha=0.3)
plt.show()

# Aplicar K-Means com k=4
optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_scaled)

# Adicionar clusters ao dataframe
df_model['cluster'] = cluster_labels

# Análise dos clusters
print(f"\n📊 Distribuição dos Clusters:")
cluster_counts = df_model['cluster'].value_counts().sort_index()
for cluster_id, count in cluster_counts.items():
    percentage = count / len(df_model) * 100
    print(f"   • Cluster {cluster_id}: {count} funcionários ({percentage:.1f}%)")

# Visualizar clusters
plt.figure(figsize=(12, 8))
scatter = plt.scatter(df_model['salario_base'], df_model['salario_liquido'], 
                     c=df_model['cluster'], cmap='viridis', alpha=0.6)
plt.xlabel('Salário Base (R$)')
plt.ylabel('Salário Líquido (R$)')
plt.title('Segmentação de Funcionários por Clusters')
plt.colorbar(scatter, label='Cluster')
plt.grid(True, alpha=0.3)
plt.show()

# Perfil dos clusters
print("\n👥 Perfil dos Clusters:")
for cluster_id in sorted(df_model['cluster'].unique()):
    cluster_data = df_model[df_model['cluster'] == cluster_id]
    print(f"\n🎯 CLUSTER {cluster_id}:")
    print(f"   • Salário Base Médio: R$ {cluster_data['salario_base'].mean():,.2f}")
    print(f"   • Salário Líquido Médio: R$ {cluster_data['salario_liquido'].mean():,.2f}")
    print(f"   • Departamento Principal: {cluster_data['departamento'].mode().iloc[0]}")
    print(f"   • Cargo Principal: {cluster_data['cargo'].mode().iloc[0]}")

## 📋 Relatório de Auditoria e Recomendações

Com base na análise exploratória e nos modelos desenvolvidos, apresentamos os principais achados e recomendações.

In [ ]:
# Gerar relatório de auditoria
print("📊 RELATÓRIO DE AUDITORIA - FOLHA DE PAGAMENTO")
print("=" * 60)

# Resumo geral
total_funcionarios = len(df_folha)
total_folha = df_folha['salario_liquido'].sum()
media_salarial = df_folha['salario_liquido'].mean()

print(f"\n📈 RESUMO EXECUTIVO:")
print(f"   • Total de Funcionários: {total_funcionarios:,}")
print(f"   • Folha de Pagamento Total: R$ {total_folha:,.2f}")
print(f"   • Salário Líquido Médio: R$ {media_salarial:,.2f}")

# Principais achados
anomalias_detectadas = len(df_model[df_model['anomalia'] == -1])
percentual_anomalias = anomalias_detectadas / total_funcionarios * 100

print(f"\n🚨 PRINCIPAIS ACHADOS:")
print(f"   • Anomalias Detectadas: {anomalias_detectadas} ({percentual_anomalias:.1f}%)")
print(f"   • Clusters Identificados: {optimal_k}")

# Top 5 funcionários com maiores salários
print(f"\n💰 TOP 5 MAIORES SALÁRIOS:")
top_salarios = df_folha.nlargest(5, 'salario_liquido')[['nome', 'cargo', 'departamento', 'salario_liquido']]
for idx, row in top_salarios.iterrows():
    print(f"   • {row['nome']}: R$ {row['salario_liquido']:,.2f} ({row['cargo']} - {row['departamento']})")

# Funcionários com anomalias (top 5)
print(f"\n⚠️  TOP 5 ANOMALIAS DETECTADAS:")
anomalias_top = df_model[df_model['anomalia'] == -1].nlargest(5, 'salario_liquido')[['nome', 'cargo', 'departamento', 'salario_liquido']]
for idx, row in anomalias_top.iterrows():
    print(f"   • {row['nome']}: R$ {row['salario_liquido']:,.2f} ({row['cargo']} - {row['departamento']})")

# Recomendações
print(f"\n💡 RECOMENDAÇÕES:")
print(f"   1. Investigar as {anomalias_detectadas} anomalias detectadas")
print(f"   2. Revisar estrutura salarial por cargo e departamento")
print(f"   3. Implementar controles automáticos para detecção de inconsistências")
print(f"   4. Estabelecer políticas claras de remuneração por cluster")
print(f"   5. Realizar auditoria mensal usando os modelos desenvolvidos")

print(f"\n✅ Relatório gerado com sucesso!")

## 🎯 Conclusões e Próximos Passos

### Principais Resultados

1. **Análise Exploratória**: Identificamos a distribuição dos salários, padrões por departamento e cargo
2. **Detecção de Anomalias**: Utilizamos Isolation Forest para identificar casos suspeitos
3. **Segmentação**: Aplicamos K-Means para criar clusters de funcionários com características similares
4. **Relatório de Auditoria**: Gerado automaticamente com achados e recomendações

### Próximos Passos

1. **Validação**: Validar as anomalias detectadas com a equipe de RH
2. **Implementação**: Integrar os modelos ao sistema de produção
3. **Monitoramento**: Criar dashboard para acompanhamento contínuo
4. **Melhoria**: Refinar os modelos com feedback da equipe
5. **Automação**: Implementar pipeline de auditoria automática

### Documentação

- Código fonte disponível neste notebook
- Modelos salvos para uso em produção
- Relatórios exportáveis em formato PDF/Excel
- Dashboards interativos em desenvolvimento

---

**📅 Última atualização**: Janeiro 2025  
**👨‍💻 Desenvolvido por**: Equipe AUDITORIA360  
**📧 Contato**: Para dúvidas ou sugestões, consulte a documentação técnica